In [8]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import pathlib
import glob
import tqdm
import cv2
import numpy as np

In [17]:
# List of sample images to process
img_path = 'sample3.jpeg'
img_list = glob.glob(img_path)
print(len(img_list))

1


In [18]:
# Load the YOLO model
docseg_model = YOLO('model/yolo-td.pt')
docseg_model.overrides['iou'] = 0.1  # NMS IoU threshold
conf_thresh = 0.75

In [23]:
def get_yolo_preds(img, docseg_model, thresh, shrink_ht = 1, shrink_wt = 1):
    image = cv2.imread(img, 0)
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # Resize the image
    height, width, _ = image.shape
    dets = []
    results = docseg_model(image, save=True, show_labels=True, show_conf=True, show_boxes=True, conf = conf_thresh)
    #results[0].save(filename = f'/home/dhruv/Projects/TD-Results/YOLO/{dataset}/{mode}/' + img.split('/')[-1])
    for entry in results:
        bboxes = entry.boxes.xyxy.numpy()
        classes = entry.boxes.cls.numpy()
        conf = entry.boxes.conf.numpy()
        for i in range(len(bboxes)):
            box = bboxes[i]
            if conf[i] > thresh:
                dets.append([box[0], box[1], box[2], box[3]])
    return dets

In [24]:
dets = get_yolo_preds(img_list[0], docseg_model, conf_thresh)
print(dets)


0: 1024x800 1 table, 269.9ms
Speed: 6.9ms preprocess, 269.9ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 800)
Results saved to runs/detect/predict3
[[54.167625, 159.30011, 741.7082, 448.96075]]


### Testing

In [59]:
import json
import glob
import tqdm

In [60]:
annot_file_path = f'/home/dhruv/Projects/TD-Dataset/{dataset}/{mode}/annotations.json'
images_path = f'/home/dhruv/Projects/TD-Dataset/{dataset}/{mode}/*.jpg'

In [61]:
fp = open(annot_file_path, 'r')
annots = json.load(fp)

In [62]:
image_annots = annots['images']
table_annots = annots['annotations']

In [63]:
image_name_list = glob.glob(images_path)
print(len(image_name_list))

360


In [64]:
def get_img_id_from_annots(image_annots, img_path):
    img_path_to_search = img_path.split('/')[-1:][0]
    for entry in image_annots:
        if img_path_to_search[:-4] in entry['file_name']:
            return entry['id']

In [65]:
def get_table_annots_from_id(table_annots, img_id):
    gt_bboxes = []
    for entry in table_annots:
        if entry['image_id'] == img_id:
            bbox = entry['bbox']
            x1 = int(bbox[0])
            y1 = int(bbox[1])
            x2 = x1 + int(bbox[2])
            y2 = y1 + int(bbox[3])
            final_bbox = [x1, y1, x2, y2]
            gt_bboxes.append(final_bbox)
    return gt_bboxes

In [68]:
def calculate_metrics(true_boxes, predicted_boxes, iou_threshold=0.9):
    TP = 0
    FP = 0
    FN = 0

    for pred_box in predicted_boxes:
        max_iou = 0
        for true_box in true_boxes:
            # Calculate Intersection over Union (IoU)
            intersection = max(0, min(pred_box[2], true_box[2]) - max(pred_box[0], true_box[0])) * \
                           max(0, min(pred_box[3], true_box[3]) - max(pred_box[1], true_box[1]))
            union = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1]) + \
                    (true_box[2] - true_box[0]) * (true_box[3] - true_box[1]) - intersection
            iou = intersection / union

            if iou > max_iou:
                max_iou = iou

        if max_iou >= iou_threshold:
            TP += 1
        else:
            FP += 1

    FN = len(true_boxes) - TP
    if FN < 0:
        FN = 0
   
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    return precision, recall

In [69]:
P = 0
R = 0
N = len(image_name_list)

for img_path in tqdm.tqdm(image_name_list):
    #print(img_path)
    img_id = get_img_id_from_annots(image_annots, img_path)
    shrink_ht = 1
    shrink_wt = 1
    gt_bboxes = get_table_annots_from_id(table_annots, img_id)
    #print(gt_bboxes)
    det_bboxes = get_yolo_preds(img_path, docseg_model, thresh = conf_thresh)
    #print(det_bboxes)
    #print('*' * 50)
    p, r = calculate_metrics(gt_bboxes, det_bboxes)
    P += p
    R += r
    

pr = P/N
rc = R/N
su = pr + rc
f1 = 2 * pr * rc / su

print(pr)
print(rc)
print(f1)

  0%|          | 0/360 [00:00<?, ?it/s]


0: 1024x800 6 tables, 307.7ms
Speed: 15.7ms preprocess, 307.7ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 800)


  0%|          | 1/360 [00:00<02:03,  2.90it/s]


0: 1024x800 (no detections), 230.4ms
Speed: 9.4ms preprocess, 230.4ms inference, 0.3ms postprocess per image at shape (1, 3, 1024, 800)


  1%|          | 2/360 [00:00<01:57,  3.04it/s]

...


0: 1024x768 1 table, 278.4ms
Speed: 7.3ms preprocess, 278.4ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 768)


100%|█████████▉| 359/360 [01:55<00:00,  3.15it/s]


0: 1024x800 2 tables, 278.2ms
Speed: 14.6ms preprocess, 278.2ms inference, 3.8ms postprocess per image at shape (1, 3, 1024, 800)


100%|██████████| 360/360 [01:56<00:00,  3.10it/s]

0.6831944444444444
0.6890277777777778
0.686098712325686
